In [ ]:
import pandas as pd

# Load metadata
metadata = pd.read_csv("data/archive/metadata.csv")

In [ ]:
cleaned_data = metadata.dropna(subset=['cord_uid', 'sha', 'title', 'abstract', 'publish_time', 'authors'])
# cleaned_data.sample(1000, random_state=73)
cleaned_data.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,xqhn0vbp,1e1286db212100993d03cc22374b624f7caee956,PMC,Airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,PMC140314,12525263.0,no-cc,"BACKGROUND: Rhinovirus, the most common cause ...",2003-01-13,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",BMC Public Health,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
1,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,PMC,Discovering human history from stomach bacteria,10.1186/gb-2003-4-5-213,PMC156578,12734001.0,no-cc,Recent analyses of human pathogens have reveal...,2003-04-28,"Disotell, Todd R",Genome Biol,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
3,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,PMC,Association of HLA class I with severe acute r...,10.1186/1471-2350-4-9,PMC212558,12969506.0,no-cc,BACKGROUND: The human leukocyte antigen (HLA) ...,2003-09-12,"Lin, Marie; Tseng, Hsiang-Kuang; Trejaut, Jean...",BMC Med Genet,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
4,0qaoam29,5b68a553a7cbbea13472721cd1ad617d42b40c26,PMC,A double epidemic model for the SARS propagation,10.1186/1471-2334-3-19,PMC222908,12964944.0,no-cc,BACKGROUND: An epidemic of a Severe Acute Resp...,2003-09-10,"Ng, Tuen Wai; Turinici, Gabriel; Danchin, Antoine",BMC Infect Dis,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
5,qj4dh6rg,3ed670f60a7be2e3e2a991ea8af1fdd5fa5e2b2c,PMC,Cloaked similarity between HIV-1 and SARS-CoV ...,10.1186/1471-2180-3-20,PMC222911,14499001.0,no-cc,BACKGROUND: Severe acute respiratory syndrome ...,2003-09-21,"Kliger, Yossef; Levanon, Erez Y",BMC Microbiol,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...


In [ ]:
import os
import json
import pandas as pd

# Path to the metadata file and the full-text directories
json_dirs = [
    "data/archive/biorxiv_medrxiv/biorxiv_medrxiv/pdf_json",
    "data/archive/comm_use_subset/comm_use_subset/pdf_json",
    "data/archive/custom_license/custom_license/pdf_json",
    "data/archive/noncomm_use_subset/noncomm_use_subset/pdf_json"
]

# Function to extract body text from JSON
def extract_body_text_from_json(file_path):
    """
    Extracts body text from a JSON file.

    Args:
        file_path (str): Path to the JSON file.

    Returns:
        str: The body text from the JSON file.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # Extract body text
        body_text = ""
        if "body_text" in data:
            body_text = " ".join([entry.get("text", "") for entry in data["body_text"]])

        return body_text
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None

# Map SHA values to JSON paths
sha_to_path = {}
for json_dir in json_dirs:
    for root, _, files in os.walk(json_dir):
        for file in files:
            if file.endswith(".json"):
                sha = os.path.splitext(file)[0]
                sha_to_path[sha] = os.path.join(root, file)

# Add body_text to the metadata
cleaned_data["body_text"] = cleaned_data["sha"].apply(lambda x: extract_body_text_from_json(sha_to_path.get(x)) if pd.notnull(x) and x in sha_to_path else None)

cleaned_data = cleaned_data.drop(columns=["source_x", "doi", "pmcid", "pubmed_id", "license", "journal", "Microsoft Academic Paper ID", "WHO #Covidence",  "has_pdf_parse", "has_pmc_xml_parse", "full_text_file", "url"])
# Save to a new CSV with the required columns
cleaned_data.to_csv("cleaned_data.csv", index=False)

print("CSV file with title, abstract, and body_text has been created successfully!")

/var/folders/dr/fn2_r29930s6nq_wqwxq_ckm0000gn/T/ipykernel_20848/1874208079.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data["body_text"] = cleaned_data["sha"].apply(lambda x: extract_body_text_from_json(sha_to_path.get(x)) if pd.notnull(x) and x in sha_to_path else None)


CSV file with title, abstract, and body_text has been created successfully!


In [ ]:
import pandas as pd

c_df = pd.read_csv("cleaned_data.csv")

# Combine 'abstract' and 'body_text' columns into a new column 'document'
c_df['document'] = c_df['abstract'] + " " + c_df['body_text']

# Display the updated DataFrame
c_df.head()

,cord_uid,sha,title,abstract,publish_time,authors,body_text,document
0,xqhn0vbp,1e1286db212100993d03cc22374b624f7caee956,Airborne rhinovirus detection and effect of ul...,"BACKGROUND: Rhinovirus, the most common cause ...",2003-01-13,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",Rhinoviruses have been associated with 40% to ...,"BACKGROUND: Rhinovirus, the most common cause ..."
1,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,Discovering human history from stomach bacteria,Recent analyses of human pathogens have reveal...,2003-04-28,"Disotell, Todd R",Charles Darwin recognized that the distributio...,Recent analyses of human pathogens have reveal...
2,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,Association of HLA class I with severe acute r...,BACKGROUND: The human leukocyte antigen (HLA) ...,2003-09-12,"Lin, Marie; Tseng, Hsiang-Kuang; Trejaut, Jean...","patient group, a further significant increase ...",BACKGROUND: The human leukocyte antigen (HLA) ...
3,0qaoam29,5b68a553a7cbbea13472721cd1ad617d42b40c26,A double epidemic model for the SARS propagation,BACKGROUND: An epidemic of a Severe Acute Resp...,2003-09-10,"Ng, Tuen Wai; Turinici, Gabriel; Danchin, Antoine",Since November 2002 (and perhaps earlier) an o...,BACKGROUND: An epidemic of a Severe Acute Resp...
4,qj4dh6rg,3ed670f60a7be2e3e2a991ea8af1fdd5fa5e2b2c,Cloaked similarity between HIV-1 and SARS-CoV ...,BACKGROUND: Severe acute respiratory syndrome ...,2003-09-21,"Kliger, Yossef; Levanon, Erez Y",Infection by many enveloped viruses requires f...,BACKGROUND: Severe acute respiratory syndrome ...


In [ ]:
# Save the updated DataFrame to a CSV file
c_df.to_csv("cleaned_data_with_document.csv", index=False)

print("DataFrame saved as 'cleaned_data_with_document.csv'")

DataFrame saved as 'cleaned_data_with_document.csv'


## entity and entity type extraction 

In [ ]:
import pandas as pd

cleaned_data = pd.read_csv("cleaned_data_with_document.csv")
documents = cleaned_data["document"].tolist()

In [ ]:
from gliner import GLiNER

model = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")

labels = [
    "virus", "disease", "symptom", "drug", "vaccine", 
    "gene", "protein", "diagnostic", "treatment", "transmission",
    "epidemiology", "public health", "host", "pathway", "immune response",
    "clinical trial", "organization", "measurement", "publication", "environment", 
    "country", "region", "city", "person", "location", "event", "method", 
]


## This just for Demo for the first document
# for document in documents[:1]:
#     entities = model.predict_entities(document, labels)
#     for entity in entities:
#         print(entity["text"], "=>", entity["label"])

        

In [ ]:
def merge_entities(entities):
    if not entities:
        return []
    merged = []
    current = entities[0]
    for next_entity in entities[1:]:
        if next_entity['label'] == current['label'] and (next_entity['start'] == current['end'] + 1 or next_entity['start'] == current['end']):
            current['text'] = text[current['start']: next_entity['end']].strip()
            current['end'] = next_entity['end']
        else:
            merged.append(current)
            current = next_entity
    # Append the last entity
    merged.append(current)
    return merged

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", r"(?<=[.?!])\s+"],                                   
                  keep_separator=False, is_separator_regex=True,
                  chunk_size=200, chunk_overlap=20)


chunks = text_splitter.split_text(documents[0])
len(chunks)

132

In [ ]:
from tqdm import tqdm

for doc_index, document in enumerate(documents[:5]):
        print(f"Processing document {doc_index + 1}/{len(documents)}...")
        
        # Split document into chunks
        chunks = text_splitter.split_text(document)

        # Initialize storage for entities
        chunks_entities = []
        entity_list = []
        duplicates = set()

        # Process each chunk
        for text in tqdm(chunks, desc=f"Processing chunks for document {doc_index}"):
            # Predict entities
            entities = model.predict_entities(text, labels, threshold=0.7)
            entities = merge_entities(entities)  # Merge overlapping/duplicate entities

            # Process entities
            chunk_entities = set()
            for entity in entities:
                chunk_entities.add(entity["text"])
                if entity["text"] in duplicates:
                    continue
                duplicates.add(entity["text"])
                entity_list.append({"text": entity["text"], "label": entity["label"]})

            # Store entities for the chunk
            chunks_entities.append(list(chunk_entities))

        # Save results for the current document
        output_file = f"entity_type_list/document_{doc_index + 1}_entities.json"
        result = {
            "document_index": doc_index,
            "chunks_entities": chunks_entities,
            "entity_list": entity_list
        }
        with open(output_file, "w") as f:
            json.dump(result, f, indent=4)
        
        print(f"Entities for document {doc_index + 1} saved to {output_file}")

In [ ]:
import csv

# Assuming entity_list is your final output
output_file = "entity_list.csv"

with open(output_file, mode="w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Entity Text", "Entity Label"])  # Header
    writer.writerows(entity_list)

print(f"Results saved to {output_file}")

Results saved to entity_list.csv


## Using Rebel For Relation Extraction

In [ ]:
from tqdm.auto import tqdm

/opt/anaconda3/envs/KG_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import pandas as pd

cleaned_data = pd.read_csv("cleaned_data_with_document.csv")
documents = cleaned_data["document"].tolist()

In [ ]:
from huggingface_hub import login

# Log in to Hugging Face Hub
login("")

In [ ]:
import json
from langchain_text_splitters import RecursiveCharacterTextSplitter
import hashlib
from transformers import AutoTokenizer

data = []  # Initialize data list
m = hashlib.md5()  # Hashing object
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def bert_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

def create_chunk_dataset(content):
    m.update(content.encode('utf-8'))
    uid = m.hexdigest()[:12]
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", r"(?<=[.?!])\s+"],                                   
        keep_separator=False, is_separator_regex=True,
        chunk_size=200, chunk_overlap=20
    )
    chunks = text_splitter.split_text(content)
    for i, chunk in enumerate(chunks):
        data.append({
            'id': f'{uid}-{i}',
            'text': chunk
        })

# Filter out NaN values from documents
filtered_documents = [doc for doc in documents[:100] if isinstance(doc, str)]

# Process your data
for document in filtered_documents:  # Assuming documents is your input data list
    create_chunk_dataset(document)

# Save chunks to a `.jsonl` file
filename = 'output_from_using_REBEL/chunks.json'
with open(filename, 'w') as outfile:
    for chunk in data:
        outfile.write(json.dumps(chunk) + '\n')

print(f"Chunks saved to {filename}")

Chunks saved to output_from_using_REBEL/chunks.json


In [ ]:
len(data)

14660

In [ ]:
def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")

In [ ]:
gen_kwargs = {
    "max_length": 256,
    "length_penalty": 0,
    "num_beams": 3,
    "num_return_sequences":1,
}

In [ ]:
len(data)

14660

In [ ]:
def generate_triples(texts):

    model_inputs = tokenizer(
        texts, max_length=512, padding=True, truncation=True, return_tensors="pt"
    )
    generated_tokens = model.generate(
        model_inputs["input_ids"].to(model.device),
        attention_mask=model_inputs["attention_mask"].to(model.device),
        **gen_kwargs
    )
    decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)
    for idx, sentence in enumerate(decoded_preds):
        et = extract_triplets(sentence)
        for t in et:
            triples.append((t["head"], t["type"], t["tail"], texts))


triples = []


for i in tqdm(range(0, len(data), 2)):
    try:
        texts = [data[i]["text"], data[i + 1]["text"]]
    except:
        texts = [data[i]["text"]]

    generate_triples(texts)

100%|██████████| 7330/7330 [3:00:17<00:00,  1.48s/it]  


In [ ]:
len(triples)

16088

In [ ]:
triples

[('Rhinovirus',
  'has effect',
  'upper respiratory tract infections',
  ['BACKGROUND: Rhinovirus, the most common cause of upper respiratory tract infections, has been implicated in asthma exacerbations and possibly asthma deaths.',
   'Although the method of transmission of rhinoviruses is disputed, several studies have demonstrated that aerosol transmission is a likely method of transmission among adults.']),
 ('aerosol transmission',
  'facet of',
  'rhinovirus',
  ['BACKGROUND: Rhinovirus, the most common cause of upper respiratory tract infections, has been implicated in asthma exacerbations and possibly asthma deaths.',
   'Although the method of transmission of rhinoviruses is disputed, several studies have demonstrated that aerosol transmission is a likely method of transmission among adults.']),
 ('detecting infectious agents',
  'studies',
  'detecting aerosolized rhinovirus',
  ['As a first step in studies of possible airborne rhinovirus transmission, we developed methods 

In [ ]:
import pandas as pd

# Convert the triples list to a DataFrame
triples_df = pd.DataFrame(triples, columns=['subject', 'relationship', 'object', 'chunk'])

# Convert the 'chunk' column to strings to make it hashable
triples_df['chunk'] = triples_df['chunk'].apply(lambda x: str(x))

# Drop rows with NaN values
triples_df = triples_df.dropna()

# Drop duplicates
triples_df_cleaned = triples_df.drop_duplicates()

# Save the cleaned DataFrame to a CSV file
triples_df_cleaned.to_csv('output_from_using_REBEL/rebel_triples.csv', index=False)

print("Triples saved to 'rebel_triples.csv' with duplicates and NaN values removed.")

Triples saved to 'rebel_triples.csv' with duplicates and NaN values removed.


In [ ]:
dsads = pd.read_csv('output_from_using_REBEL/rebel_triples.csv')
dsads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15866 entries, 0 to 15865
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   subject       15866 non-null  object
 1   relationship  15866 non-null  object
 2   object        15864 non-null  object
 3   chunk         15866 non-null  object
dtypes: object(4)
memory usage: 495.9+ KB


## Json to CSV

In [ ]:
import json
import hashlib
from transformers import AutoTokenizer
from langchain_text_splitters import RecursiveCharacterTextSplitter
from tqdm import tqdm
import pandas as pd
import time
import os

# Initialize tokenizer and text splitter
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", r"(?<=[.?!])\s+"],
    keep_separator=False, is_separator_regex=True,
    chunk_size=200, chunk_overlap=20
)

# Load metadata
metadata_df = pd.read_csv("cleaned_data_with_document.csv")
metadata_map = metadata_df.to_dict("index")

# Initialize hashing object
m = hashlib.md5()

# Function to extract triples (placeholder for your actual logic)
def extract_triples(text):
    file_path = "rebel_triples.json"
    if not os.path.exists(file_path):
        print(f"Error: {file_path} not found.")
        return []

    # Load the source JSON file
    with open(file_path, "r") as f:
        source_data = json.load(f)

    # Process the data into the target format
    converted_data = []
    for triple in source_data:
        # Ensure the triple is a list with three elements
        if len(triple) == 3:
            subject, relationship, obj = triple
            converted_data.append({
                "subject": subject,
                "relationship": relationship,
                "object": obj
            })
    return converted_data

# Process each document
for doc_index, document in enumerate(documents[:2]):  # Iterate over the first two documents
    m.update(document.encode('utf-8'))
    uid = m.hexdigest()[:12]
    chunks = text_splitter.split_text(document)  # Split the document into chunks
    
    # Initialize storage for chunks
    doc_chunks = []
    
    # Process each chunk
    for i, chunk in enumerate(tqdm(chunks, desc=f"Processing chunks for document {doc_index}")):
        doc_chunks.append({
            'id': f'{uid}-{i}',
            'text': chunk
        })

    # Save results for the current document
    output_file = f"document_{doc_index + 1}_chunks.json"
    result = {
        "document_index": doc_index,
        "document_id": uid,
        "chunks": doc_chunks
    }
    with open(output_file, "w") as f:
        json.dump(result, f, indent=4)

    print(f"Chunks for document {doc_index + 1} saved to {output_file}")

    # Extract triples and enrich with metadata
    errors = []
    all_triples = []
    for i in tqdm(range(len(doc_chunks))):
        try:
            text = doc_chunks[i]['text']
            chunk_id = doc_chunks[i]['id']
            triples = extract_triples(text)  # Extract triples

            # Get the metadata for the current document
            document_metadata = metadata_map[doc_index]

            # Enrich each triple with the corresponding metadata and chunk information
            for triple in triples:
                enriched_triple = {
                    "subject": triple["subject"],
                    "relationship": triple["relationship"],
                    "object": triple["object"],
                    # Metadata
                    # "cord_uid": document_metadata["cord_uid"],
                    # "sha": document_metadata["sha"],
                    # "title": document_metadata["title"],
                    # "abstract": document_metadata["abstract"],
                    # "publish_time": document_metadata["publish_time"],
                    # "authors": document_metadata["authors"],
                    # Chunk information
                    "chunk_id": chunk_id,
                    "chunk_text": text
                }
                all_triples.append(enriched_triple)
            # time.sleep(3)  # Avoid rate-limiting by adding a delay
        except Exception as e:
            print(f"Error for chunk {i}: {e}")
            errors.append({"chunk": i, "error": str(e)})
            all_triples.append([])  # Append an empty list for failed chunks

    # Save the enriched triples to a JSON file
    output_file = f"all_triples_with_metadata_{doc_index}.json"
    with open(output_file, "w") as f:
        json.dump(all_triples, f, indent=4)
    print(f"Output saved to {output_file}")

# Optionally save errors to a separate file
if errors:
    with open("errors.json", "w") as f:
        json.dump(errors, f, indent=4)
    print("Errors saved to 'errors.json'.")

Processing chunks for document 0: 100%|██████████| 132/132 [00:00<00:00, 1693113.54it/s]


Chunks for document 1 saved to document_1_chunks.json


100%|██████████| 132/132 [00:00<00:00, 4266.31it/s]


Output saved to all_triples_with_metadata_0.json


Processing chunks for document 1: 100%|██████████| 73/73 [00:00<00:00, 1515763.33it/s]


Chunks for document 2 saved to document_2_chunks.json


100%|██████████| 73/73 [00:00<00:00, 3021.53it/s]

Output saved to all_triples_with_metadata_1.json


In [ ]:
import json
import pandas as pd

# Function to convert JSON to a list of dictionaries
def json_to_list(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    # Flatten the JSON structure
    flattened_data = []
    for doc in data:
        for triple in doc:
            flattened_data.append(triple)
    
    return flattened_data

# Initialize an empty list to store all data
all_data = []

# Convert each JSON file to a list of dictionaries and append to all_data
for doc_index in range(2):  # Assuming you have two documents
    json_file = f"output_from_using_REBEL/all_triples_with_metadata_{doc_index}.json"
    all_data.extend(json_to_list(json_file))

# Convert the combined data to a DataFrame
df = pd.DataFrame(all_data)

# Save the DataFrame to a single CSV file
csv_file = "all_triples_with_metadata_combined.csv"
df.to_csv(csv_file, index=False)
print(f"Combined data saved to {csv_file}")


Combined data saved to all_triples_with_metadata_combined.csv


In [ ]:
import pandas as pd

# Load the CSV file with the combined data
all_data = pd.read_csv("output_from_using_REBEL/REBEL_all_triples_with_metadata_combined.csv")
all_data.describe()

,subject,relationship,object,cord_uid,sha,title,abstract,publish_time,authors
count,128535,128535,128535,128535,128535,128535,128535,128535,128535
unique,466,52,402,2,2,2,2,2,2
top,SARS,subclass of,SARS,xqhn0vbp,1e1286db212100993d03cc22374b624f7caee956,Airborne rhinovirus detection and effect of ul...,"BACKGROUND: Rhinovirus, the most common cause ...",2003-01-13,"Myatt, Theodore A; Johnston, Sebastian L; Rudn..."
freq,3690,27470,4920,82764,82764,82764,82764,82764,82764


In [ ]:
all_data.shape

(128535, 9)

In [ ]:
print(all_data[['subject', 'relationship', 'object']].drop_duplicates().shape)

(627, 3)


In [ ]:
all_data_non_duplicates = all_data[['subject', 'relationship', 'object']].drop_duplicates()

# Print the output to the console
all_data_non_duplicates.to_csv('output_from_using_REBEL/REBEL_all_triples_No_duplicate.csv', index=False)

## Entity Type Adding Process

In [ ]:
import pandas as pd

# Load the CSV file with the triples
triples_df = pd.read_csv('output_from_using_REBEL/rebel_triples.csv')

# Function to convert a triple into a sentence
def triple_to_sentence(triple):
    return f"{triple['subject']} {triple['relationship']} {triple['object']}."

# Apply the function to each row in the DataFrame
triples_df['sentence'] = triples_df.apply(triple_to_sentence, axis=1)

# Display the resulting DataFrame with sentences
triples_df.head()

# Save the sentences to a new CSV file
triples_df.to_csv('output_from_using_REBEL/rebel_triples_sentences.csv', index=False)
print("Sentences saved to 'output_from_using_REBEL/rebel_triples_sentences.csv'")

Sentences saved to 'output_from_using_REBEL/rebel_triples_sentences.csv'


In [ ]:
triples_sen = pd.read_csv("output_from_using_REBEL/rebel_triples_sentences.csv")
triples_sen

In [ ]:
from gliner import GLiNER

model = GLiNER.from_pretrained("gliner-community/gliner_large-v2.5")

triples_sen = pd.read_csv("output_from_using_REBEL/rebel_triples_sentences.csv")

text = triples_sen["sentence"].tolist()

labels = [
    "virus", "disease", "symptom", "drug", "vaccine", 
    "gene", "diagnostic", "treatment", "transmission",
    "epidemiology", "public health", "host", "immune response",
    "organization", "measurement", "publication", "environment", "substance", 
    "molecule", "chemical", "bacteria", "country", "region", "city", "province", 
    "state", "location", "event", "method", "person", "people"
]

entity_label_dict = {}

# Iterate through the sentences and predict entities
for sentence in text:
    entities = model.predict_entities(sentence, labels, threshold=0.7)
    for entity in entities:
        entity_text = entity["text"]
        entity_label = entity["label"]
        # Store the entity and label in the dictionary
        entity_label_dict[entity_text] = entity_label


Fetching 10 files: 100%|██████████| 10/10 [00:00<00:00, 248183.67it/s]
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
print(len((entity_label_dict)))
entity_label_dict

In [ ]:
import pandas as pd

# Convert the dictionary to a DataFrame
entity_label_df = pd.DataFrame(list(entity_label_dict.items()), columns=['Entity Text', 'Entity Label'])

# Save the DataFrame to a CSV file
entity_label_df.to_csv('output_from_using_REBEL/entity_label_dict.csv', index=False)

print("Entity label dictionary saved to 'entity_label_dict.csv'")

Entity label dictionary saved to 'entity_label_dict.csv'


In [ ]:
triples_df = pd.read_csv('output_from_using_REBEL/rebel_triples_sentences.csv')
triples_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15866 entries, 0 to 15865
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   subject       15866 non-null  object
 1   relationship  15866 non-null  object
 2   object        15864 non-null  object
 3   chunk         15866 non-null  object
 4   sentence      15866 non-null  object
dtypes: object(5)
memory usage: 619.9+ KB


In [ ]:
import pandas as pd

# Load the triples and entity label data
triples_df = pd.read_csv('output_from_using_REBEL/rebel_triples_sentences.csv')
entity_label_df = pd.read_csv('output_from_using_REBEL/entity_label_dict.csv')

# Create a dictionary from the entity label DataFrame
entity_label_dict = {row["Entity Text"].strip().lower(): row["Entity Label"].strip() for _, row in entity_label_df.iterrows()}

# Function to get the entity label
def get_entity_label(entity):
    return entity_label_dict.get(entity.strip().lower(), 'unknown')

# Fill NaN values with an empty string
triples_df['subject'] = triples_df['subject'].fillna('')
triples_df['object'] = triples_df['object'].fillna('')

# Add the labels to the triples DataFrame
triples_df['subject_label'] = triples_df['subject'].apply(get_entity_label)
triples_df['object_label'] = triples_df['object'].apply(get_entity_label)

# Save the updated DataFrame to a new CSV file
triples_df.to_csv('output_from_using_REBEL/rebel_triples_labels.csv', index=False)

print("Updated triples with labels saved to 'output_from_using_REBEL/rebel_triples_labels.csv'")

Updated triples with labels saved to 'output_from_using_REBEL/rebel_triples_labels.csv'


In [ ]:
import pandas as pd
triples_labels = pd.read_csv("output_from_using_REBEL/rebel_triples_labels.csv")
triples_labels

,subject,relationship,object,chunk,sentence,subject_label,object_label
0,Rhinovirus,has effect,upper respiratory tract infections,"['BACKGROUND: Rhinovirus, the most common caus...",Rhinovirus has effect upper respiratory tract ...,virus,disease
1,aerosol transmission,facet of,rhinovirus,"['BACKGROUND: Rhinovirus, the most common caus...",aerosol transmission facet of rhinovirus.,transmission,virus
2,detecting infectious agents,studies,detecting aerosolized rhinovirus,['As a first step in studies of possible airbo...,detecting infectious agents studies detecting ...,unknown,unknown
3,aerosol chamber,use,aerosol,['As a first step in studies of possible airbo...,aerosol chamber use aerosol.,unknown,environment
4,rhinoviral,subclass of,aerosols,['To determine the effect of UV irradiation on...,rhinoviral subclass of aerosols.,unknown,substance
...,...,...,...,...,...,...,...
15861,GeoDRM,facet of,Web-based geospatial technologies,['Such developments are of interest to the geo...,GeoDRM facet of Web-based geospatial technolog...,method,method
15862,OGC GeoDRM Working Group,field of work,digital rights management,['The mission of OGC GeoDRM Working Group is t...,OGC GeoDRM Working Group field of work digital...,organization,method
15863,Google,product or material produced,maps,['The mission of OGC GeoDRM Working Group is t...,Google product or material produced maps.,organization,location
15864,map,part of,visual language of geography,['Building on the powerful and universal visua...,map part of visual language of geography.,environment,unknown


In [ ]:
triples_labels.describe()

,subject,relationship,object,chunk,sentence,subject_label,object_label
count,15866,15866,15864,15866,15866,15866,15866
unique,8384,107,6630,7325,13686,32,32
top,genome,subclass of,protein,['To identify RBD amino acid residues that mig...,John F. Kennedy School of Government part of H...,unknown,unknown
freq,236,4554,281,12,39,4934,3379


## LLM refinement

In [ ]:
from langchain_ollama.chat_models import ChatOllama
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from tqdm import tqdm

# Initialize the Ollama model
llm = ChatOllama(model="qwen2.5", temperature=0, format="json")


# Function to predict the label using LLM
def predict_label(text, chunk):
    system_message_template = """
        Predict the label for the following entity based on the given context. 
        The label must be chosen strictly from the predefined list below. Return a JSON object, 
        strictly formatted as valid JSON. Do not add any text, explanations, or comments outside the JSON.

        Labels: [
            "virus", "disease", "symptom", "drug", "vaccine", 
            "gene", "protein", "diagnostic", "treatment", "transmission",
            "epidemiology", "public health", "host", "pathway", "immune response",
            "clinical trial", "organization", "measurement", "publication", "environment", 
            "country", "region", "city", "person", "location", "event", "method"
            ]

        For example: 
        [
            {{"entity": "Eiffel Tower", "label": "location"}},
            {{"entity": "Albert Einstein", "label": "person"}},
            {{"entity": "COVID-19", "label": "virus"}}
        ]
        Output must be valid JSON with no additional text.

        Context: \n\n{chunk}
        Entity: \n\n{entity} 
    """
    system_message = system_message_template.format(entity=text, chunk=chunk)
    user_message = f"Entity: {text}\n\nLabel:"

    response = llm(
        [SystemMessage(content=system_message), HumanMessage(content=user_message)]
    )
    return response.content.strip()


# Filter rows with unknown labels
unknown_subjects = triples_labels[triples_labels["subject_label"] == "unknown"]
unknown_objects = triples_labels[triples_labels["object_label"] == "unknown"]

subject_labels = []
object_labels = []

# Predict labels for unknown subjects
for index, row in tqdm(
    unknown_subjects.iterrows(),
    total=unknown_subjects.shape[0],
    desc="Predicting labels for unknown subjects",
):
    predicted_label = predict_label(row["subject"], row["chunk"])
    subject_labels.append(predicted_label)

# Predict labels for unknown objects
for index, row in tqdm(
    unknown_objects.iterrows(),
    total=unknown_objects.shape[0],
    desc="Predicting labels for unknown objects",
):
    predicted_label = predict_label(row["object"], row["chunk"])
    object_labels.append(predicted_label)

Predicting labels for unknown subjects:   0%|          | 0/4934 [00:00<?, ?it/s]/var/folders/dr/fn2_r29930s6nq_wqwxq_ckm0000gn/T/ipykernel_7900/664957814.py:39: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(
Predicting labels for unknown objects: 100%|██████████| 3379/3379 [1:18:40<00:00,  1.40s/it]


In [ ]:
import json

# Convert each string in the subject_labels list to a dictionary
subject_labels_dicts = [json.loads(label) for label in subject_labels]

# Convert each string in the object_labels list to a dictionary
object_labels_dicts = [json.loads(label) for label in object_labels]

# Print the converted lists of dictionaries
print("Subject Labels Dicts:", subject_labels_dicts)
print("Object Labels Dicts:", object_labels_dicts)
print(type(subject_labels_dicts[0]["entity"]))
print(type(object_labels_dicts[0]))

Subject Labels Dicts: [{'entity': 'detecting infectious agents', 'label': 'method'}, {'entity': 'aerosol chamber', 'label': 'method'}, {'entity': 'rhinoviral', 'label': 'virus'}, {'entity': 'TCID(50)', 'label': 'measurement'}, {'entity': 'germicidal', 'label': 'virus'}, {'entity': 'infectious viral particles', 'label': 'virus'}, {'entity': 'autumn outbreak period', 'label': 'event'}, {'entity': 'one experimental study', 'label': 'publication'}, {'entity': 'ultraviolet germicidal irradiation', 'label': 'method'}, {'entity': 'bacilli Calmette-Guérin', 'label': 'vaccine'}, {'entity': 'drying jar', 'label': 'device'}, {'entity': 'Collison', 'label': 'device'}, {'entity': 'UV exposure chamber', 'label': 'equipment'}, {'entity': 'filters', 'label': 'method'}, {'entity': 'Teflon filter', 'label': 'diagnostic'}, {'entity': 'bypass filter', 'label': 'device'}, {'entity': 'semi-nested', 'label': 'method'}, {'entity': 'spiked', 'label': 'vaccine'}, {'entity': 'single pass air', 'label': 'environm

In [ ]:
import json

# Save the dictionaries to separate JSON files
with open('output_from_using_REBEL/subject_labels.json', 'w') as f:
    json.dump(subject_labels_dicts, f, indent=4)

with open('output_from_using_REBEL/object_labels.json', 'w') as f:
    json.dump(object_labels_dicts, f, indent=4)

print("Subject labels saved to 'output_from_using_REBEL/subject_labels.json'")
print("Object labels saved to 'output_from_using_REBEL/object_labels.json'")


Subject labels saved to 'output_from_using_REBEL/subject_labels.json'
Object labels saved to 'output_from_using_REBEL/object_labels.json'


In [ ]:
import pandas as pd
import json

# Load the triples and entity label data
triples_df = pd.read_csv('output_from_using_REBEL/rebel_triples_labels.csv')

# Load the subject and object labels from JSON files
with open('output_from_using_REBEL/subject_labels.json', 'r') as f:
    subject_labels_dicts = json.load(f)

with open('output_from_using_REBEL/object_labels.json', 'r') as f:
    object_labels_dicts = json.load(f)

# Create dictionaries from the subject and object labels
subject_label_dict = {entry['entity'].strip().lower(): entry['label'].strip() for entry in subject_labels_dicts if 'entity' in entry}
object_label_dict = {entry['entity'].strip().lower(): entry['label'].strip() for entry in object_labels_dicts if 'entity' in entry}

# Function to get the entity label
def get_entity_label(entity, label_dict):
    return label_dict.get(entity.strip().lower(), 'unknown')

# Fill NaN values with an empty string
triples_df['subject'] = triples_df['subject'].fillna('')
triples_df['object'] = triples_df['object'].fillna('')

# Update the unknown labels in the triples DataFrame
triples_df['subject_label'] = triples_df.apply(
    lambda row: get_entity_label(row['subject'], subject_label_dict) if row['subject_label'] == 'unknown' else row['subject_label'],
    axis=1
)

triples_df['object_label'] = triples_df.apply(
    lambda row: get_entity_label(row['object'], object_label_dict) if row['object_label'] == 'unknown' else row['object_label'],
    axis=1
)
# Remove rows with NaN values in 'subject' or 'object'
triples_df = triples_df.dropna(subset=['subject', 'object'])
# Reorder the columns to have subject_label and object_label next to the triples
columns_order = ["subject","relationship","object","subject_label","object_label","sentence","chunk"]
triples_df = triples_df[columns_order]

# Save the updated DataFrame to a new CSV file
triples_df.to_csv('output_from_using_REBEL/Final_rebel_triples_labels_large.csv', index=False)

print("Updated triples with matched labels saved to 'output_from_using_REBEL/Final_rebel_triples_labels.csv'")

Updated triples with matched labels saved to 'output_from_using_REBEL/Final_rebel_triples_labels.csv'


In [22]:
import pandas as pd
from tqdm import tqdm

# Load the data
Final_data = pd.read_csv("output_from_using_REBEL/Final_rebel_triples_labels_large.csv")

# Convert subject and object to lowercase for consistency
Final_data['subject'] = Final_data['subject'].str.lower()
Final_data['object'] = Final_data['object'].str.lower()

# Add these lines before processing to create the initial subject_chunk and object_chunk columns
Final_data['subject_chunk'] = Final_data['chunk']
Final_data['object_chunk'] = Final_data['chunk']

In [7]:
import pandas as pd
import ast
import re


def clean_text(text):
    """Remove only single and double quotes from a string, keeping other special characters."""
    return re.sub(r"[\'\"]", '', text)  # Remove only ' and "

def combine_chunks_with_labels(df):
    """Combine chunks for each entity while keeping labels."""
    combined_chunks = {}

    for _, row in df.iterrows():
        subject, object_ = row['subject'], row['object']
        subject_label, object_label = row['subject_label'], row['object_label']
        chunk = row['chunk']

        # Convert chunk from string representation to actual list
        try:
            chunk = ast.literal_eval(chunk) if isinstance(chunk, str) else chunk
        except (SyntaxError, ValueError):
            chunk = []  

        # Ensure chunk is a list
        if not isinstance(chunk, list):
            chunk = [chunk]  

        # Clean chunks by removing special characters (except ())
        cleaned_chunk = [clean_text(item) for item in chunk if isinstance(item, str)]
        
        # Store subject with its label
        if subject not in combined_chunks:
            combined_chunks[subject] = {"label": subject_label, "chunks": set()}
        combined_chunks[subject]["chunks"].update(cleaned_chunk)

        # Store object with its label
        if object_ not in combined_chunks:
            combined_chunks[object_] = {"label": object_label, "chunks": set()}
        combined_chunks[object_]["chunks"].update(cleaned_chunk)

    # Convert sets to lists for easier handling
    for key in combined_chunks:
        combined_chunks[key]["chunks"] = list(combined_chunks[key]["chunks"])

    return combined_chunks

# Load DataFrame
df = pd.read_csv("output_from_using_REBEL/Final_rebel_triples_labels_large.csv")

# Convert subject and object to lowercase for consistency
df['subject'] = df['subject'].str.lower()
df['object'] = df['object'].str.lower()

# Combine chunks while keeping labels
combined_chunks = combine_chunks_with_labels(df)

# Convert dictionary to DataFrame for saving
combined_df = pd.DataFrame([
    {"entity": key, "label": value["label"], "chunks": value["chunks"]}
    for key, value in combined_chunks.items()
])

# Save to CSV
combined_df.to_csv("combined_chunks_with_labels.csv", index=False)

In [8]:
combined_df[combined_df['entity'] == 'replication cycle']["chunks"][7477]

['Recombination rates between adjacent markers are large, on the order of 0.05 to 0.1.',
 'As shown, the results were not affected qualitatively, and only slightly quantitatively.',
 'Assuming that ten viral replication cycles occurred during our experiment—based on data on the timing of coat protein detection—the per base and replication cycle recombination rate was on the order of 2 × 10(−5) to 4 × 10(−5).',
 'This first determination of a virus recombination rate during a single multi-cellular host infection indicates that recombination is very frequent in the everyday life of this virus.',
 'In case this assumption is incorrect, we did calculations assuming five, seven, ten, or 20 replication cycles during these 21 d.',
 'The inferred per generation recombination and interference rates, assuming that CaMV undergoes ten replication cycles during the 21 d between infection and sampling, are given for each of the ten plants in Table 2 .']

In [27]:
import pandas as pd

# Load entity-chunk mapping
ents_chunks_df = pd.read_csv("combined_chunks.csv")

# Convert ents_chunks to a dictionary for quick lookup
ents_chunks_dict = dict(zip(ents_chunks_df['entity'], ents_chunks_df['chunks']))

# Load the final data
Final_data = pd.read_csv("output_from_using_REBEL/Final_rebel_triples_labels_large.csv")

# Convert subject and object to lowercase to ensure consistent matching
Final_data['subject'] = Final_data['subject'].str.lower()
Final_data['object'] = Final_data['object'].str.lower()

# Initialize subject_chunk and object_chunk
Final_data['subject_chunk'] = Final_data['subject'].map(ents_chunks_dict)
Final_data['object_chunk'] = Final_data['object'].map(ents_chunks_dict)

# Fill missing values with empty lists
Final_data['subject_chunk'] = Final_data['subject_chunk'].apply(lambda x: eval(x) if isinstance(x, str) else [])
Final_data['object_chunk'] = Final_data['object_chunk'].apply(lambda x: eval(x) if isinstance(x, str) else [])

Final_data


,subject,relationship,object,subject_label,object_label,sentence,chunk,subject_chunk,object_chunk
0,rhinovirus,has effect,upper respiratory tract infections,virus,disease,Rhinovirus has effect upper respiratory tract ...,"['BACKGROUND: Rhinovirus, the most common caus...","([In this work, we have shown that several hal...","[BACKGROUND: Rhinovirus, the most common cause..."
1,aerosol transmission,facet of,rhinovirus,transmission,virus,aerosol transmission facet of rhinovirus.,"['BACKGROUND: Rhinovirus, the most common caus...","[BACKGROUND: Rhinovirus, the most common cause...","([In this work, we have shown that several hal..."
2,detecting infectious agents,studies,detecting aerosolized rhinovirus,method,method,detecting infectious agents studies detecting ...,['As a first step in studies of possible airbo...,[As a first step in studies of possible airbor...,[As a first step in studies of possible airbor...
3,aerosol chamber,use,aerosol,method,environment,aerosol chamber use aerosol.,['As a first step in studies of possible airbo...,[As a first step in studies of possible airbor...,"([Using rhinovirus type 15, they inoculated vo..."
4,rhinoviral,subclass of,aerosols,virus,substance,rhinoviral subclass of aerosols.,['To determine the effect of UV irradiation on...,[To determine the effect of UV irradiation on ...,"([Using rhinovirus type 15, they inoculated vo..."
...,...,...,...,...,...,...,...,...,...
15861,geodrm,facet of,web-based geospatial technologies,method,method,GeoDRM facet of Web-based geospatial technolog...,['Such developments are of interest to the geo...,[Such developments are of interest to the geos...,[Such developments are of interest to the geos...
15862,ogc geodrm working group,field of work,digital rights management,organization,method,OGC GeoDRM Working Group field of work digital...,['The mission of OGC GeoDRM Working Group is t...,[The mission of OGC GeoDRM Working Group is to...,[The mission of OGC GeoDRM Working Group is to...
15863,google,product or material produced,maps,organization,location,Google product or material produced maps.,['The mission of OGC GeoDRM Working Group is t...,([Google released Google Maps (http://maps.goo...,([The mission of OGC GeoDRM Working Group is t...
15864,map,part of,visual language of geography,environment,method,map part of visual language of geography.,['Building on the powerful and universal visua...,([One can display data collected at smaller ge...,[Building on the powerful and universal visual...


In [28]:
Final_data.to_csv("updated_final_data.csv", index=False)

In [42]:
updated_final_data = pd.read_csv("updated_final_data.csv")

In [46]:
updated_final_data["subject_chunk"][0]

'([\'In this work, we have shown that several hallmarks of cellular autophagosomes, including the localization of GFP-LC3 into discrete punctate structures and staining with MDC followed by fixation, can be observed in human cells whether they are treated with tamoxifen or infected with poliovirus or rhinovirus.\', \'Furthermore, we have employed a new criterion for autophagy, the colocalization of LC3 and LAMP1, to demonstrate that this intracellular rearrangement was observed in cells treated with tamoxifen, infected with picornaviruses (poliovirus, rhinovirus 2, or rhinovirus 14) or transfected with plasmids that express poliovirus proteins 2BC and 3A (see Figures 2 and 8) .\'], [\'For cryofixation and EM analysis, H1-HeLa cells were grown in EMEM supplemented with 0.2 M HEPES and 0.1 M MgCl 2 in flasks.\', \'Cells were infected with virus at an MOI of 50 PFU/cell for poliovirus or 50 TCID 50 /cell for rhinovirus, then washed three times with PBS, trypsinized, and collected by centr

In [56]:
type(updated_final_data["subject_chunk"][0][0])

list

In [47]:
import ast

# Convert back to a list
updated_final_data["subject_chunk"] = updated_final_data["subject_chunk"].apply(ast.literal_eval)


In [ ]:
import pandas as pd
import ast  # Safely evaluate tuple and list strings
import json  # Store embeddings as JSON
import concurrent.futures
from langchain_ollama import OllamaEmbeddings
from tqdm import tqdm  # Progress bar

# Initialize Ollama Embeddings Model
embedding_model = OllamaEmbeddings(model="nomic-embed-text:latest")

# Load the CSV file
df = pd.read_csv("updated_final_data.csv")

# Function to process and embed chunks
def process_chunks(chunk_data):
    if pd.isna(chunk_data):  # Skip NaN values
        return json.dumps([])  # Store as JSON empty list

    # Convert string representation of tuple/list to Python object
    try:
        chunk_data = ast.literal_eval(chunk_data)
    except (ValueError, SyntaxError):
        return json.dumps([])  # Store as JSON empty list

    # Ensure chunk_data is a list (flatten tuple if necessary)
    if isinstance(chunk_data, tuple):
        chunk_list = list(chunk_data)  # Convert tuple to list
    elif isinstance(chunk_data, list):
        chunk_list = chunk_data  # Use as is
    else:
        return json.dumps([])  # If not a list/tuple, return empty list

    # Flatten lists inside tuples (extract only valid strings)
    flat_chunk_list = []
    for item in chunk_list:
        if isinstance(item, list):  # Nested lists inside tuple or list
            for sub_item in item:
                if isinstance(sub_item, str):
                    flat_chunk_list.append(sub_item)  # Extract string
        elif isinstance(item, str):
            flat_chunk_list.append(item)  # Directly add strings

    return flat_chunk_list  # Return the clean list of text chunks


# Apply processing to extract clean chunk lists
df["subject_chunk"] = df["subject_chunk"].apply(process_chunks)
df["object_chunk"] = df["object_chunk"].apply(process_chunks)

# Flatten all text chunks into one list for batch embedding
all_texts = list(set(sum(df["subject_chunk"].tolist(), []) + sum(df["object_chunk"].tolist(), [])))

# Function to generate embeddings in parallel
def embed_texts(text_list):
    try:
        return {text: embedding_model.embed_query(text) for text in text_list}
    except Exception as e:
        print(f"Error embedding batch: {e}")
        return {}

# Split text list into chunks for parallel processing
batch_size = 256  # Adjust batch size based on GPU memory
text_batches = [all_texts[i : i + batch_size] for i in range(0, len(all_texts), batch_size)]

# Parallel embedding processing
embedding_results = {}
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(tqdm(executor.map(embed_texts, text_batches), total=len(text_batches)))

# Merge results into a dictionary
for result in results:
    embedding_results.update(result)

# Function to replace chunk lists with embeddings
def replace_with_embeddings(chunk_list):
    return json.dumps([{ "chunk": text, "embedding": embedding_results.get(text, []) } for text in chunk_list])

# Replace subject and object chunks with their embeddings
df["subject_chunk_embeddings"] = df["subject_chunk"].apply(replace_with_embeddings)
df["object_chunk_embeddings"] = df["object_chunk"].apply(replace_with_embeddings)

# Save the updated dataframe to CSV
df.to_csv("updated_final_data_with_embeddings.csv", index=False)

print("Processing complete. CSV saved as 'updated_final_data_with_embeddings.csv'.")


100%|██████████| 58/58 [02:32<00:00,  2.63s/it]


Processing complete. CSV saved as 'updated_final_data_with_embeddings.csv'.


In [ ]:
import pandas as pd
df_final_embed = pd.read_csv("updated_final_data_with_embeddings.csv", engine='c')

In [ ]:
# Drop the specified columns
df = df_final_embed.drop(columns=["chunk", "subject_chunk", "object_chunk"])

# Save the updated dataframe to CSV
# df.to_csv("updated_final_data_with_embeddings.csv", index=False)

In [15]:
df

,subject,relationship,object,subject_label,object_label,sentence,subject_chunk_embeddings,object_chunk_embeddings
0,rhinovirus,has effect,upper respiratory tract infections,virus,disease,Rhinovirus has effect upper respiratory tract ...,"[{""chunk"": ""In this work, we have shown that s...","[{""chunk"": ""BACKGROUND: Rhinovirus, the most c..."
1,aerosol transmission,facet of,rhinovirus,transmission,virus,aerosol transmission facet of rhinovirus.,"[{""chunk"": ""BACKGROUND: Rhinovirus, the most c...","[{""chunk"": ""In this work, we have shown that s..."
2,detecting infectious agents,studies,detecting aerosolized rhinovirus,method,method,detecting infectious agents studies detecting ...,"[{""chunk"": ""As a first step in studies of poss...","[{""chunk"": ""As a first step in studies of poss..."
3,aerosol chamber,use,aerosol,method,environment,aerosol chamber use aerosol.,"[{""chunk"": ""As a first step in studies of poss...","[{""chunk"": ""Using rhinovirus type 15, they ino..."
4,rhinoviral,subclass of,aerosols,virus,substance,rhinoviral subclass of aerosols.,"[{""chunk"": ""To determine the effect of UV irra...","[{""chunk"": ""Using rhinovirus type 15, they ino..."
...,...,...,...,...,...,...,...,...
15857,geodrm,facet of,web-based geospatial technologies,method,method,GeoDRM facet of Web-based geospatial technolog...,"[{""chunk"": ""Such developments are of interest ...","[{""chunk"": ""Such developments are of interest ..."
15858,ogc geodrm working group,field of work,digital rights management,organization,method,OGC GeoDRM Working Group field of work digital...,"[{""chunk"": ""The mission of OGC GeoDRM Working ...","[{""chunk"": ""The mission of OGC GeoDRM Working ..."
15859,google,product or material produced,maps,organization,location,Google product or material produced maps.,"[{""chunk"": ""Google released Google Maps http:m...","[{""chunk"": ""The mission of OGC GeoDRM Working ..."
15860,map,part of,visual language of geography,environment,method,map part of visual language of geography.,"[{""chunk"": ""One can display data collected at ...","[{""chunk"": ""Building on the powerful and unive..."


In [ ]:
import ast  # Safely evaluate tuple and list strings

chunk_data = ast.literal_eval(df["subject_chunk_embeddings"][0])
chunk_data

In [19]:
import pandas as pd

df = pd.read_csv("updated_final_data_with_embeddings.csv")

In [ ]:
import ast
from neo4j import GraphDatabase

# Initialize Neo4j driver (adjust URI, user, password as needed)
uri = "bolt://localhost:7687"
user = "neo4j"
password = "password"
driver = GraphDatabase.driver(uri, auth=(user, password))

def create_nodes_and_relationships(tx, row):
    subject = row['subject']
    object_ = row['object']
    subject_label = row['subject_label']
    object_label = row['object_label']
    relationship = row['relationship']

    # Escape labels/relationship
    subject_label_escaped = f"`{subject_label.replace('`', '')}`"
    object_label_escaped  = f"`{object_label.replace('`', '')}`"
    relationship_escaped  = f"`{relationship.replace('`', '')}`"

    query_main = f"""
    MERGE (s:Entity:{subject_label_escaped} {{name: $subjectName}})
    MERGE (o:Entity:{object_label_escaped} {{name: $objectName}})
    MERGE (s)-[r:{relationship_escaped} {{name: $relationship}}]->(o)
    """

    tx.run(query_main, subjectName=subject, objectName=object_, relationship=relationship)

    # Then do the same approach for chunk nodes...
    # subject_chunk_list = ast.literal_eval(row['subject_chunk_embeddings'])
    # etc...
    
    # 2) Create Chunk nodes for the subject
    #    'subject_chunk_embeddings' is a JSON-like string with a list of dicts.
    subject_chunk_list = ast.literal_eval(row['subject_chunk_embeddings'])
    for chunk_dict in subject_chunk_list:
        chunk_text = chunk_dict.get('chunk', '')
        embedding = chunk_dict.get('embedding', [])
        
        query_sub_chunk = f"""
        MATCH (s:Entity:{subject_label} {{name: $subjectName}})
        MERGE (chunkNode:Chunk {{chunk: $chunkText}})
        SET chunkNode.embedding = $embedding
        MERGE (s)-[:HAS_CHUNK]->(chunkNode)
        """
        tx.run(query_sub_chunk,
               subjectName=subject,
               chunkText=chunk_text,
               embedding=embedding)
    
    # 3) Create Chunk nodes for the object
    object_chunk_list = ast.literal_eval(row['object_chunk_embeddings'])
    for chunk_dict in object_chunk_list:
        chunk_text = chunk_dict.get('chunk', '')
        embedding = chunk_dict.get('embedding', [])
        
        query_obj_chunk = f"""
        MATCH (o:Entity:{object_label} {{name: $objectName}})
        MERGE (chunkNode:Chunk {{chunk: $chunkText}})
        SET chunkNode.embedding = $embedding
        MERGE (o)-[:HAS_CHUNK]->(chunkNode)
        """
        tx.run(query_obj_chunk,
               objectName=object_,
               chunkText=chunk_text,
               embedding=embedding)

# -----------------------
# Process the DataFrame
# -----------------------
with driver.session() as session:
    for idx, row in df.iterrows():
        session.write_transaction(create_nodes_and_relationships, row)

driver.close()
print("Data import to Neo4j completed.")

### Count Labels

In [ ]:
import pandas as pd

# Load the CSV file

# Extract unique labels from 'subject_label' and 'object_label' columns
subject_labels = Final_data['subject_label'].unique().tolist()
object_labels = Final_data['object_label'].unique().tolist()

# Combine and deduplicate the labels
all_labels = list(set(subject_labels + object_labels))

# Print the list of labels
print(all_labels)
print(len(all_labels))

['language', 'forensic', 'occupation', 'sample', 'mutation', 'infrastructure', 'vaccination', 'vocabulary_word', 'condition', 'stem', 'drug', 'contact', 'word', 'effect', 'procedure', 'policy', 'term', 'vulnerability', 'question', 'bacteria', 'service', 'anatomy', 'signature', 'section', 'information', 'scenario', 'symptom', 'media', 'cell', 'tool', 'viral barcodes', 'virus transport medium', 'object', 'list', 'person', 'organ', 'public health', 'method', 'data', 'technology', 'vessel', 'organization', 'pathway', 'cell line', 'region', 'country', 'primer', 'function', 'device', 'dataset', 'algorithm', 'city', 'disease', 'definition', 'activity', 'phase', 'growth phase', 'process', 'value', 'number', 'pattern', 'idea', 'database', 'ontology', 'sequence polymorphism', 'file', 'genotype', 'vocabulary', 'state', 'diagnosis', 'product', 'cell type', 'environment', 'ethical issue', 'cell cycle stage', 'relation', 'vaccine', 'variable', 'contact tracing', 'season', 'marker', 'viral cell cultu